# Async Operations and Background Tasks

This notebook covers async operations in FastAPI:
- Async vs sync endpoints
- File uploads with UploadFile
- Background tasks for processing
- Streaming responses
- Concurrent request handling

FastAPI is built on ASGI (async) which enables high-performance concurrent request handling.

## Setup

In [ ]:
from fastapi import FastAPI, File, Form, UploadFile, BackgroundTasks, HTTPException, status
from fastapi.responses import StreamingResponse, JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, Field, EmailStr
from typing import List, Optional, Generator
import asyncio
import aiofiles
import time
from datetime import datetime
import json
import io
from pathlib import Path
import tempfile

## 1. Async vs Sync Endpoints

FastAPI supports both async and sync endpoints. Understanding when to use each is important for performance.

In [ ]:
app = FastAPI(
    title="Async Operations API",
    description="Demonstrating async operations, file uploads, and background tasks",
    version="1.0.0"
)

# Sync endpoint - runs in threadpool
@app.get("/sync/sleep/{seconds}")
def sync_sleep(seconds: int):
    """Sync endpoint - blocks the thread"""
    time.sleep(seconds)
    return {
        "type": "sync",
        "slept_for": seconds,
        "message": f"Slept for {seconds} seconds (blocking)"
    }

# Async endpoint - non-blocking
@app.get("/async/sleep/{seconds}")
async def async_sleep(seconds: int):
    """Async endpoint - non-blocking"""
    await asyncio.sleep(seconds)
    return {
        "type": "async",
        "slept_for": seconds,
        "message": f"Slept for {seconds} seconds (non-blocking)"
    }

# Async endpoint with external API call (simulated)
@app.get("/async/fetch")
async def fetch_external_data(delay: int = 1):
    """Async endpoint making external HTTP request (simulated)"""
    start_time = time.time()
    
    # Simulate async HTTP request
    await asyncio.sleep(delay)
    data = {"status": "success", "delay": delay}
    
    elapsed = (time.time() - start_time) * 1000
    
    return {
        "message": "Fetched data successfully",
        "elapsed_ms": round(elapsed, 2),
        "data": data
    }

# CPU-bound operation - use sync
@app.get("/sync/compute")
def compute_fibonacci(n: int = 30):
    """CPU-bound operation - better as sync"""
    def fib(x):
        if x <= 1:
            return x
        return fib(x-1) + fib(x-2)
    
    start_time = time.time()
    result = fib(n)
    elapsed = (time.time() - start_time) * 1000
    
    return {
        "n": n,
        "result": result,
        "elapsed_ms": round(elapsed, 2),
        "note": "CPU-bound, runs in threadpool"
    }

print("Async vs Sync endpoints added!")

## 2. File Upload Endpoints

FastAPI provides excellent support for file uploads using `UploadFile`.

In [ ]:
# Temporary directory for uploaded files
UPLOAD_DIR = Path(tempfile.gettempdir()) / "fastapi_uploads"
UPLOAD_DIR.mkdir(exist_ok=True)

class FileUploadResponse(BaseModel):
    filename: str
    content_type: str
    size_bytes: int
    saved_to: str
    upload_time_ms: float

# Single file upload
@app.post("/upload/file", response_model=FileUploadResponse)
async def upload_file(file: UploadFile = File(...)):
    """Upload a single file"""
    start_time = time.time()
    
    # Read file content
    content = await file.read()
    file_size = len(content)
    
    # Save file
    file_path = UPLOAD_DIR / file.filename
    async with aiofiles.open(file_path, 'wb') as f:
        await f.write(content)
    
    elapsed = (time.time() - start_time) * 1000
    
    return {
        "filename": file.filename,
        "content_type": file.content_type,
        "size_bytes": file_size,
        "saved_to": str(file_path),
        "upload_time_ms": round(elapsed, 2)
    }

# Multiple file upload
@app.post("/upload/files")
async def upload_multiple_files(files: List[UploadFile] = File(...)):
    """Upload multiple files"""
    start_time = time.time()
    results = []
    
    for file in files:
        content = await file.read()
        file_size = len(content)
        
        file_path = UPLOAD_DIR / file.filename
        async with aiofiles.open(file_path, 'wb') as f:
            await f.write(content)
        
        results.append({
            "filename": file.filename,
            "size_bytes": file_size,
            "content_type": file.content_type
        })
    
    elapsed = (time.time() - start_time) * 1000
    
    return {
        "files_uploaded": len(results),
        "total_size_bytes": sum(r["size_bytes"] for r in results),
        "upload_time_ms": round(elapsed, 2),
        "files": results
    }

# Upload with metadata
class FileMetadata(BaseModel):
    description: str
    tags: List[str] = []
    category: str

@app.post("/upload/with-metadata")
async def upload_with_metadata(
    file: UploadFile = File(...),
    description: str = Form(...),
    tags: str = Form(default=""),  # Comma-separated
    category: str = Form(...)
):
    """Upload file with metadata"""
    content = await file.read()
    
    file_path = UPLOAD_DIR / file.filename
    async with aiofiles.open(file_path, 'wb') as f:
        await f.write(content)
    
    # Save metadata
    metadata = {
        "filename": file.filename,
        "description": description,
        "tags": [t.strip() for t in tags.split(",") if t.strip()],
        "category": category,
        "uploaded_at": datetime.now().isoformat(),
        "size_bytes": len(content)
    }
    
    metadata_path = UPLOAD_DIR / f"{file.filename}.json"
    async with aiofiles.open(metadata_path, 'w') as f:
        await f.write(json.dumps(metadata, indent=2))
    
    return metadata

print("File upload endpoints added!")

## 3. Background Tasks

Background tasks run after the response is sent to the client.

In [ ]:
# Simulated task log
task_log = []

def log_task(message: str):
    """Background task: log message with timestamp"""
    timestamp = datetime.now().isoformat()
    log_entry = f"[{timestamp}] {message}"
    task_log.append(log_entry)
    print(log_entry)

async def send_email_notification(email: str, subject: str, message: str):
    """Background task: simulate sending email"""
    await asyncio.sleep(2)  # Simulate email sending
    log_entry = f"Email sent to {email}: {subject}"
    task_log.append(log_entry)
    print(log_entry)

async def process_file_in_background(file_path: str, filename: str):
    """Background task: process uploaded file"""
    await asyncio.sleep(3)  # Simulate processing
    log_entry = f"File processed: {filename}"
    task_log.append(log_entry)
    print(log_entry)

# Endpoint with background task
class UserRegistration(BaseModel):
    username: str
    email: EmailStr
    full_name: str

@app.post("/users/register")
async def register_user(user: UserRegistration, background_tasks: BackgroundTasks):
    """Register user and send welcome email in background"""
    # Add background tasks
    background_tasks.add_task(
        log_task,
        f"User registered: {user.username}"
    )
    background_tasks.add_task(
        send_email_notification,
        user.email,
        "Welcome!",
        f"Welcome to our service, {user.full_name}!"
    )
    
    # Return immediately (background tasks run after response)
    return {
        "message": "Registration successful",
        "username": user.username,
        "email": user.email,
        "note": "Welcome email will be sent shortly"
    }

# File upload with background processing
@app.post("/upload/process")
async def upload_and_process(file: UploadFile, background_tasks: BackgroundTasks):
    """Upload file and process in background"""
    # Save file
    content = await file.read()
    file_path = UPLOAD_DIR / file.filename
    
    async with aiofiles.open(file_path, 'wb') as f:
        await f.write(content)
    
    # Add background processing task
    background_tasks.add_task(
        process_file_in_background,
        str(file_path),
        file.filename
    )
    
    return {
        "message": "File uploaded successfully",
        "filename": file.filename,
        "size_bytes": len(content),
        "status": "processing in background"
    }

# View task log
@app.get("/tasks/log")
async def get_task_log():
    """Get background task log"""
    return {
        "total_tasks": len(task_log),
        "log": task_log[-20:]  # Last 20 entries
    }

print("Background task endpoints added!")

## 4. Streaming Responses

Stream data to clients as it becomes available.

In [ ]:
# Generator function for streaming
async def generate_numbers(n: int):
    """Generate numbers with delay"""
    for i in range(n):
        await asyncio.sleep(0.1)  # Simulate computation
        yield f"Number: {i}\n"

async def generate_log_stream():
    """Generate simulated log stream"""
    log_messages = [
        "Starting application...",
        "Loading configuration...",
        "Connecting to database...",
        "Database connected",
        "Loading models...",
        "Models loaded",
        "Application ready",
    ]
    
    for msg in log_messages:
        await asyncio.sleep(0.5)
        timestamp = datetime.now().strftime("%H:%M:%S")
        yield f"[{timestamp}] {msg}\n"

# Streaming endpoint
@app.get("/stream/numbers")
async def stream_numbers(count: int = 10):
    """Stream numbers to client"""
    return StreamingResponse(
        generate_numbers(count),
        media_type="text/plain"
    )

@app.get("/stream/logs")
async def stream_logs():
    """Stream simulated logs"""
    return StreamingResponse(
        generate_log_stream(),
        media_type="text/plain"
    )

# Stream JSON data
async def generate_json_stream(items: int):
    """Generate JSON objects"""
    for i in range(items):
        await asyncio.sleep(0.2)
        data = {
            "id": i,
            "timestamp": datetime.now().isoformat(),
            "value": i * 10
        }
        yield json.dumps(data) + "\n"

@app.get("/stream/json")
async def stream_json(items: int = 5):
    """Stream JSON objects (newline-delimited JSON)"""
    return StreamingResponse(
        generate_json_stream(items),
        media_type="application/x-ndjson"
    )

# Simulate LLM streaming
async def generate_llm_response(prompt: str):
    """Simulate LLM token streaming"""
    response = f"This is a response to: '{prompt}'. The answer involves multiple tokens being streamed."
    words = response.split()
    
    for word in words:
        await asyncio.sleep(0.1)  # Simulate token generation
        yield word + " "

@app.get("/stream/llm")
async def stream_llm(prompt: str = "What is FastAPI?"):
    """Stream LLM-style response"""
    return StreamingResponse(
        generate_llm_response(prompt),
        media_type="text/plain"
    )

print("Streaming endpoints added!")

## 5. Concurrent Request Handling

Demonstrate FastAPI's ability to handle concurrent requests efficiently.

In [ ]:
# Track concurrent requests
active_requests = {"count": 0}
request_history = []

@app.get("/concurrent/slow")
async def slow_endpoint(delay: int = 2, request_id: str = "unknown"):
    """Slow endpoint for testing concurrency"""
    active_requests["count"] += 1
    max_concurrent = active_requests["count"]
    
    start_time = time.time()
    await asyncio.sleep(delay)
    elapsed = time.time() - start_time
    
    active_requests["count"] -= 1
    
    result = {
        "request_id": request_id,
        "delay": delay,
        "elapsed": round(elapsed, 2),
        "concurrent_requests_during_processing": max_concurrent
    }
    
    request_history.append(result)
    return result

@app.get("/concurrent/stats")
async def concurrent_stats():
    """Get concurrency statistics"""
    return {
        "current_active": active_requests["count"],
        "total_processed": len(request_history),
        "recent_requests": request_history[-10:]
    }

print("Concurrent handling endpoints added!")

## 6. Running the Server

In [ ]:
# Create TestClient
client = TestClient(app)

print(f"\n✅ Client ready for testing")
print(f"📚 The app would normally run at http://127.0.0.1:8000/docs in production")

## 7. Testing Async Operations

In [ ]:
# Test 1: Async vs Sync sleep
print("Test 1 - Async vs Sync:")

# Async endpoint
start = time.time()
response = client.get("/async/sleep/1")
async_time = time.time() - start
print(f"Async sleep: {async_time:.2f}s")

# Sync endpoint
start = time.time()
response = client.get("/sync/sleep/1")
sync_time = time.time() - start
print(f"Sync sleep: {sync_time:.2f}s")
print()

In [ ]:
# Test 2: File upload
print("Test 2 - File upload:")

# Create test file
file_content = b"This is a test file for FastAPI upload demonstration."
files = {"file": ("test.txt", file_content, "text/plain")}

response = client.post("/upload/file", files=files)
result = response.json()
print(f"Uploaded: {result['filename']}")
print(f"Size: {result['size_bytes']} bytes")
print(f"Upload time: {result['upload_time_ms']:.2f}ms")
print()

In [ ]:
# Test 3: Multiple file upload
print("Test 3 - Multiple file upload:")

files = [
    ("files", ("file1.txt", b"Content of file 1", "text/plain")),
    ("files", ("file2.txt", b"Content of file 2", "text/plain")),
    ("files", ("file3.txt", b"Content of file 3", "text/plain")),
]

response = client.post("/upload/files", files=files)
result = response.json()
print(f"Files uploaded: {result['files_uploaded']}")
print(f"Total size: {result['total_size_bytes']} bytes")
print(f"Upload time: {result['upload_time_ms']:.2f}ms")
print()

In [ ]:
# Test 4: Background tasks
print("Test 4 - Background tasks:")

user_data = {
    "username": "testuser",
    "email": "test@example.com",
    "full_name": "Test User"
}

response = client.post("/users/register", json=user_data)
result = response.json()
print(f"Response: {result['message']}")
print(f"Note: {result['note']}")
print("\nWaiting for background tasks to complete...")
time.sleep(3)

# Check task log
response = client.get("/tasks/log")
log_data = response.json()
print(f"\nTask log ({log_data['total_tasks']} tasks):")
for entry in log_data['log'][-3:]:
    print(f"  {entry}")
print()

In [ ]:
# Test 5: Streaming response
print("Test 5 - Streaming response:")

with client.stream("GET", "/stream/numbers?count=5") as response:
    print("Receiving stream:")
    for line in response.iter_lines():
        print(f"  {line}")
print()

In [ ]:
# Test 6: Concurrent requests
print("Test 6 - Concurrent requests:")
import concurrent.futures

def make_request(request_id):
    response = client.get(f"/concurrent/slow?delay=2&request_id=req_{request_id}")
    return response.json()

# Make 5 concurrent requests
print("Making 5 concurrent requests (2s delay each)...")
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(make_request, i) for i in range(5)]
    results = [f.result() for f in concurrent.futures.as_completed(futures)]

total_time = time.time() - start_time

print(f"\nTotal time: {total_time:.2f}s (should be ~2s, not 10s!)")
print("This demonstrates async concurrency - all requests processed simultaneously")

# Get stats
response = client.get("/concurrent/stats")
stats = response.json()
print(f"\nTotal requests processed: {stats['total_processed']}")
print()

In [ ]:
# Test 7: File upload with background processing
print("Test 7 - File upload with background processing:")

file_content = b"Large file content that needs processing..."
files = {"file": ("process_me.txt", file_content, "text/plain")}

response = client.post("/upload/process", files=files)
result = response.json()
print(f"Response: {result['message']}")
print(f"Status: {result['status']}")
print("\nResponse returned immediately, processing in background...")
print("Waiting 4 seconds for background processing...")
time.sleep(4)

# Check task log
response = client.get("/tasks/log")
log_data = response.json()
print(f"\nRecent tasks:")
for entry in log_data['log'][-2:]:
    print(f"  {entry}")
print()

## 8. Key Takeaways

### What we learned:

1. **Async vs Sync**:
   - Use `async def` for I/O-bound operations (HTTP requests, file I/O, database)
   - Use regular `def` for CPU-bound operations (runs in threadpool)
   - Async endpoints don't block other requests
   - Use `await` for async operations, never use blocking `time.sleep()`

2. **File Uploads**:
   - `UploadFile` provides efficient file upload handling
   - Supports single and multiple file uploads
   - Use `aiofiles` for async file I/O
   - Can combine file uploads with other form data

3. **Background Tasks**:
   - Use `BackgroundTasks` for operations after response
   - Good for: sending emails, logging, processing files
   - Tasks run after response is sent (faster response)
   - Multiple tasks can be added to a single response

4. **Streaming Responses**:
   - Use `StreamingResponse` for large data or real-time updates
   - Generator functions with `yield` produce stream
   - Good for: logs, LLM responses, large files
   - Client receives data as it's generated

5. **Concurrency**:
   - FastAPI handles concurrent requests efficiently
   - Async endpoints can process multiple requests simultaneously
   - 5 requests with 2s delay = ~2s total (not 10s!)
   - No need for manual thread/process management

### Best Practices:
- Use async for I/O-bound operations
- Use sync for CPU-bound operations
- Offload long tasks to background tasks
- Stream large responses instead of loading in memory
- Set appropriate timeouts for async operations
- Clean up uploaded files periodically

### Next steps:
- In the next notebook, we'll cover authentication and security
- We'll implement API keys, JWT tokens, rate limiting, and CORS

In [ ]:
print(f"\n🎉 Congratulations! You've completed Async Operations!\n")
print(f"Client ready for testing!")
print(f"In production, run: uvicorn app:app --host 0.0.0.0 --port 8000")